In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm

# Function to get image paths and labels
def get_image_paths_and_labels(root_dir):
    image_paths = []
    labels = []
    for subset in ['train', 'test', 'val']:
        for label in ['Paper', 'Rock', 'Scissors']:
            folder_path = os.path.join(root_dir, f'RPS_{subset}', f'{label}_{subset}')
            for image_file in os.listdir(folder_path):
                if image_file.endswith('.png'):
                    image_paths.append(os.path.join(folder_path, image_file))
                    labels.append(label)
    return image_paths, labels


In [5]:
# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Load datasets and create data loaders
root_directory = "E:/NYCU/Signal & Image Process Data/Police txt & RPS Dataset/RPS dataset"
train_paths, train_labels = get_image_paths_and_labels(root_directory)
train_dataset = ImageFolder(root=os.path.join(root_directory, 'RPS_train'), transform=transform)
val_dataset = ImageFolder(root=os.path.join(root_directory, 'RPS_val'), transform=transform)
test_dataset = ImageFolder(root=os.path.join(root_directory, 'RPS_test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


In [6]:
# Define the model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # 3 output classes for Rock, Paper, Scissors

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Device definition
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
# Train the model
num_epochs = 5
best_val_accuracy = 0.0
best_model_weights = None

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f'Training Loss: {epoch_loss:.4f}')


Training Loss: 0.3488


Training Loss: 0.0354


Training Loss: 0.0165


Training Loss: 0.0099


Training Loss: 0.0068


In [8]:
# Validate the model and save the best model
for epoch in range(num_epochs):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
    print(f'Validation Accuracy: {val_accuracy:.4f}')

    # Save the best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_weights = model.state_dict()

        # Save the best model weights
        torch.save(best_model_weights, 'best_model_weights.pth')


Validation Accuracy: 1.0000
Validation Accuracy: 1.0000
Validation Accuracy: 1.0000
Validation Accuracy: 1.0000
Validation Accuracy: 1.0000


In [9]:
# Evaluate the model on test data
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_accuracy = correct / total
print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 1.0000


In [10]:
# Reload the best model weights
if best_model_weights is not None:
    model.load_state_dict(best_model_weights)

# Test the model with reloaded weights
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
reloaded_test_accuracy = correct / total
print(f'Reloaded Test Accuracy: {reloaded_test_accuracy:.4f}')


Reloaded Test Accuracy: 1.0000
